# Intro to Object Detection Colab

Welcome to the object detection colab!  This demo will take you through the steps of running an "out-of-the-box" detection model on a collection of images.

## Imports and Setup

In [5]:
!pip install  tensorflow-gpu==2.2.0

  Using cached https://files.pythonhosted.org/packages/ef/e6/6a7be21491b528790e733b4bb3d2707f630fef3ee890a1aebaba1639c1e6/tensorflow_gpu-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1d/74/0a6fcb206dcc72a6da9a62dd81784bfdbff5fedb099982861dc2219014fb/tensorboard-2.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/f5/926ae53d6a226ec0fda5208e0e581cffed895ccc89e36ba76a8e60895b78/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
     |████████████████████████████████| 2.9MB 6.7MB/s 
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.32.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflo

In [1]:
import tensorflow as tf 
print(tf.__version__)
tf.test.gpu_device_name()

2.2.0


'/device:GPU:0'

In [ ]:
# # import tensorflow as tf 
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2650, done.
remote: Counting objects: 100% (2650/2650), done.
remote: Compressing objects: 100% (2203/2203), done.
remote: Total 2650 (delta 671), reused 1299 (delta 415), pack-reused 0
Receiving objects: 100% (2650/2650), 32.62 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (671/671), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Utilities

In [4]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

In [5]:
# @title Choose the model to use, then evaluate the cell.
MODELS = {'centernet_with_keypoints': 'centernet_hg104_512x512_kpts_coco17_tpu-32', 'centernet_without_keypoints': 'centernet_hg104_512x512_coco17_tpu-8'}

model_display_name = 'centernet_without_keypoints' # @param ['centernet_with_keypoints', 'centernet_without_keypoints']
model_name = MODELS[model_display_name]

### Build a detection model and load pre-trained model weights

This sometimes takes a little while, please be patient!

In [6]:
# Download the checkpoint and put it into models/research/object_detection/test_data/

if model_display_name == 'centernet_with_keypoints':
  !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
  !tar -xf centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
  !mv centernet_hg104_512x512_kpts_coco17_tpu-32/checkpoint models/research/object_detection/test_data/
else:
  !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_coco17_tpu-8.tar.gz
  !tar -xf centernet_hg104_512x512_coco17_tpu-8.tar.gz
  !mv centernet_hg104_512x512_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2021-05-24 06:09:13--  http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.197.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426100886 (1.3G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’

centernet_hg104_512 100%[===================>]   1.33G   161MB/s    in 8.5s    

2021-05-24 06:09:22 (161 MB/s) - ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’ saved [1426100886/1426100886]



In [7]:
import os 
pipeline_config = os.path.join('models/research/object_detection/configs/tf2/',
                                'centernet_hourglass104_512x512_coco17_tpu-8' + '.config')
model_dir = 'models/research/object_detection/test_data/checkpoint/'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

# Load label map data (for plotting).

Label maps correspond index numbers to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [8]:
label_map_path = '/content/mscoco_label_map.pbtxt'#configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

### Putting everything together!

Run the below code which loads an image, runs it through the detection model and visualizes the detection results, including the keypoints.

Note that this will take a long time (several minutes) the first time you run this code due to tf.function's trace-compilation --- on subsequent runs (e.g. on new images), things will be faster.

Here are some simple things to try out if you are curious:
* Try running inference on your own images (local paths work)
* Modify some of the input images and see if detection still works.  Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
* Print out `detections['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
* Set min_score_thresh to other values (between 0 and 1) to allow more detections in or to filter out more detections.

Note that you can run this cell repeatedly without rerunning earlier cells.


In [ ]:
image_dir = '/content/test_images'
# for i in file
image_path = os.path.join(image_dir, '1786.jpg')
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
  keypoints = detections['detection_keypoints'][0].numpy()
  keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=get_keypoint_tuples(configs['eval_config']))

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

In [ ]:
image_np.shape

In [10]:
# keypoints = detections['detection_keypoints'][0].numpy()
# keypoints
def get_results(image_np, frame_no):
  input_tensor = tf.convert_to_tensor(
  np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)
  im_height,im_width  = image_np.shape[0],image_np.shape[1]
  boxes = detections['detection_boxes'][0].numpy()
  # detections['detection_classes'][0].numpy()+1
  score = detections['detection_scores'][0].numpy()
  cls = detections['detection_classes'][0].numpy() + 1
  # print(im_width, im_height)
  
  # img = image_np.copy()
  import cv2 
  lst =[]
  for j,i in enumerate(score):
    if i> 0.3:
      # if cls[j] not in [37,39]:
      #   continue
      # print(cls[j], i)
      ymin, xmin, ymax, xmax= boxes[j].tolist()
      x1,x2,y1,y2  = (xmin * im_width, xmax * im_width,
                                    ymin * im_height, ymax * im_height)
      x1,y1,x2,y2 = int(x1),int(y1), int(x2), int(y2)
      lst.append([x1,y1,x2,y2,cls[j],frame_no])
      cv2.rectangle(image_np, (x1,y1), (x2,y2), (0,255,0),2)
  if frame_no%100==0:
    print(frame_no, len(lst))
  
      # x,y,
      # print(x1,x2,y1,y2)
      
  return lst, image_np
  # cv2.imwrite('output.jpg',img )
# plt.figure(figsize=(400,800))
# plt.imshow(img)


In [31]:
with open('files.txt','r') as f:
  content= f.readlines()
print(len(content))
content =[int(i[:-5]) for  i in content]
content[:10]

348


[1222, 1224, 1226, 1228, 1238, 1242, 1244, 1246, 1306, 1310]

In [11]:
import cv2

video = cv2.VideoCapture("/content/drive/MyDrive/cmatch.mp4");
# video.set(cv2.CAP_PROP_FPS, int(20))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('results.mp4',fourcc, 20, (1280,720))
f=open('tf_output.json', 'a+')


# if __name__ == '__main__':
    # print("Frame rate : {0}".format(video.get(cv2.CAP_PROP_FPS)))
k=0
cor =[]
while video.isOpened():
    ret1, frame2 = video.read()
    # cv2.imshow("Changed", frame2)

    
    if ret1:
      
        # b = cv2.resize(frame2,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        # try:
        # if k in content:
        lst,frame =get_results(frame2,k)
        if len(cor)==0 and len(lst)>0:
          cor = lst 
        elif len(lst)>0:
          # if len(lst)
          cor+=lst
        
        f.write(str(lst))
        print(k, len(lst))
        # print(len(lst), k)
        k+=1
        if k >3000:
          break
        # cv2.imshow("Changed", b)
        # if cv2.waitKey(30) & 0xFF == ord('q') or k>10:  # press q to quit
        #     break
        b = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break

video.release()
out.release()
# import pandas as pd
# pd.to_csv(cor).to_csv('/content/drive/MyDrive/tf_ou.csv')
# cv2.destroyAllWindows()

In [17]:
import pandas as pd 
# pd.DataFrame(cor).to_csv('/content/drive/MyDrive/tf_output.csv')

In [19]:
data = pd.DataFrame(cor)
data.values[1]

array([None, None, None, None, None, None, None, None, None, None, None],
      dtype=object)

In [ ]:
sum((data.values!=None))

In [ ]:
import cv2

video = cv2.VideoCapture("/content/drive/MyDrive/cmatch.mp4");
# video.set(cv2.CAP_PROP_FPS, int(20))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('results.mp4',fourcc, 20, (1280,720))
f=open('tf_output.json', 'a+')


# if __name__ == '__main__':
    # print("Frame rate : {0}".format(video.get(cv2.CAP_PROP_FPS)))
k=0
cor =[]
while video.isOpened():
    ret1, frame2 = video.read()
    # cv2.imshow("Changed", frame2)

    
    if ret1:
      
        # b = cv2.resize(frame2,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        # try:
        # if k in content:
        lst,frame =get_results(frame2,k)
        if len(cor)==0:
          cor = lst 
        else:
          cor+=lst
        
        f.write(str(lst))
        print(k, len(lst))
        # print(len(lst), k)
        k+=1
        if k >3000:
          break
        # cv2.imshow("Changed", b)
        # if cv2.waitKey(30) & 0xFF == ord('q') or k>10:  # press q to quit
        #     break
        b = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break

video.release()
out.release()
import pandas as pd
pd.to_csv(cor).to_csv('/content/drive/MyDrive/tf_output.csv')
# cv2.destroyAllWindows()

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!mv results.mp4  /content/drive/MyDrive/res_obj/results.mp4

In [19]:
import json 
with open('/content/drive/MyDrive/cricket_cor.json', 'w') as f:
  json.dump(str(cor),f ,indent=2)

## Digging into the model's intermediate predictions

For this part we will assume that the detection model is a CenterNet model following Zhou et al (https://arxiv.org/abs/1904.07850).  And more specifically, we will assume that `detection_model` is of type `meta_architectures.center_net_meta_arch.CenterNetMetaArch`.

As one of its intermediate predictions, CenterNet produces a heatmap of box centers for each class (for example, it will produce a heatmap whose size is proportional to that of the image that lights up at the center of each, e.g., "zebra"). In the following, we will visualize these intermediate class center heatmap predictions.

In [ ]:
if detection_model.__class__.__name__ != 'CenterNetMetaArch':
  raise AssertionError('The meta-architecture for this section '
  'is assumed to be CenterNetMetaArch!')

def get_heatmap(predictions_dict, class_name):
  """Grabs class center logits and apply inverse logit transform.

  Args:
    predictions_dict: dictionary of tensors containing a `object_center`
      field of shape [1, heatmap_width, heatmap_height, num_classes]
    class_name: string name of category (e.g., `horse`)

  Returns:
    heatmap: 2d Tensor heatmap representing heatmap of centers for a given class
      (For CenterNet, this is 128x128 or 256x256) with values in [0,1]
  """
  class_index = label_map_dict[class_name]
  class_center_logits = predictions_dict['object_center'][0]
  class_center_logits = class_center_logits[0][
    :, :, class_index - label_id_offset]
  heatmap = tf.exp(class_center_logits) / (tf.exp(class_center_logits) + 1)
  return heatmap

def unpad_heatmap(heatmap, image_np):
  """Reshapes/unpads heatmap appropriately.

  Reshapes/unpads heatmap appropriately to match image_np.

  Args:
    heatmap: Output of `get_heatmap`, a 2d Tensor
    image_np: uint8 numpy array with shape (img_height, img_width, 3).  Note
      that due to padding, the relationship between img_height and img_width
      might not be a simple scaling.

  Returns:
    resized_heatmap_unpadded: a resized heatmap (2d Tensor) that is the same
      size as `image_np`
  """
  heatmap = tf.tile(tf.expand_dims(heatmap, 2), [1, 1, 3]) * 255
  pre_strided_size = detection_model._stride * heatmap.shape[0]
  resized_heatmap = tf.image.resize(
      heatmap, [pre_strided_size, pre_strided_size],
      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  resized_heatmap_unpadded = tf.slice(resized_heatmap, begin=[0,0,0], size=shapes)
  return tf.image.resize(
      resized_heatmap_unpadded,
      [image_np.shape[0], image_np.shape[1]],
      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)[:,:,0]


class_name = 'kite'
heatmap = get_heatmap(predictions_dict, class_name)
resized_heatmap_unpadded = unpad_heatmap(heatmap, image_np)
plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.imshow(resized_heatmap_unpadded, alpha=0.7,vmin=0, vmax=160, cmap='viridis')
plt.title('Object center heatmap (class: ' + class_name + ')')
plt.show()

class_name = 'person'
heatmap = get_heatmap(predictions_dict, class_name)
resized_heatmap_unpadded = unpad_heatmap(heatmap, image_np)
plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.imshow(resized_heatmap_unpadded, alpha=0.7,vmin=0, vmax=160, cmap='viridis')
plt.title('Object center heatmap (class: ' + class_name + ')')
plt.show()